In [ ]:
import random
# 1. Извлечение Текста из Презентации
# Функция для извлечения текста из презентации (предполагается, что она уже реализована)
from pptx import Presentation
from pptx.exc import PackageNotFoundError
def extract_text_from_presentation(file_path):
    '''
    Эта функция extract_text_from_presentation делает следующее:

    * Открывает файл презентации по указанному пути.
    * Проходит по всем слайдам и фигурам внутри них.
    * Извлекает текст из текстовых полей.
    * Собирает весь текст в один список и возвращает его в виде одной строки.
    '''
    text_runs = []
    try:
        prs = Presentation(file_path)
        for slide in prs.slides:
            for shape in slide.shapes:
                if not shape.has_text_frame:
                    continue
                for paragraph in shape.text_frame.paragraphs:
                    for run in paragraph.runs:
                        text_runs.append(run.text)
    except PackageNotFoundError:
        print(f"Файл {file_path} не найден.")
    except Exception as e:
        print(f"Произошла ошибка при чтении файла: {e}")
    return ' '.join(text_runs)
    

# Пример использования функции
file_path = 'your_presentation.pptx'
presentation_text = extract_text_from_presentation(file_path)

# 2. Анализ Содержимого (Оптимизация выбора ключевого слова)
#-----------------------------------------------------------------------------

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from random import choice

# Убедитесь, что NLTK установлен и загружены необходимые ресурсы
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


def select_key_word(sentence):
    '''
    Выбирает ключевое слово из предложения, исключая стоп-слова и выбирая существительные, глаголы или прилагательные.
    '''
    # Токенизация и определение частей речи
    words = word_tokenize(sentence)
    tagged_words = pos_tag(words)

    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    significant_words = [word for word, tag in tagged_words if word.lower() not in stop_words and tag in ['NN', 'VB', 'JJ']]

    # Выбор ключевого слова
    if significant_words:
        return choice(significant_words)
    else:
        # Если значимых слов нет, выбираем случайное слово
        return choice(words)
# 2.1 Генерация Вопросов

def create_quiz_questions(text, num_questions=5):
    # Разделение текста на предложения
    sentences = sent_tokenize(text)
    questions = []
    # Генерация вопросов
    for i in range(num_questions):
        # Выбор случайного предложения для вопроса
        sentence = sentences[i % len(sentences)]
        
        # Выбор ключевого слова для замены на пропуск
        answer = select_key_word(sentence)
        words = sentence.split()
        words[words.index(answer)] = '_____'

        # Формулирование вопроса
        question = ' '.join(words)
        questions.append({'question': question, 'answer': answer})

    return questions

# Пример использования функции
extracted_text = presentation_text
quiz_questions = create_quiz_questions(extracted_text)
for q in quiz_questions:
    print(f"Вопрос: {q['question']}")
    print(f"Ответ: {q['answer']}\n")
#----------------------------------------------------------------------------
# 3. Генерация неправильных ответов

from nltk.corpus import wordnet
from random import sample

# Убедитесь, что NLTK и WordNet установлены и загружены необходимые ресурсы
nltk.download('wordnet')

def get_semantically_similar_words(word, num_words=3):
    '''
    Возвращает список слов, семантически близких к заданному слову.
    '''
    synonyms = set()
    antonyms = set()

    # Получение синонимов и антонимов для слова
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
            if lemma.antonyms():
                antonyms.add(lemma.antonyms()[0].name())

    # Объединение синонимов и антонимов и удаление исходного слова
    similar_words = list(synonyms.union(antonyms))
    if word in similar_words:
        similar_words.remove(word)

    # Выбор случайного набора слов
    if len(similar_words) >= num_words:
        return sample(similar_words, num_words)
    else:
        return similar_words  # Возвращает меньше слов, если не хватает синонимов/антонимов

def create_answers(correct_answer, num_incorrect_answers=3):
    '''
    Создает список неправильных ответов, семантически близких к правильному ответу.
    '''
    incorrect_answers = get_semantically_similar_words(correct_answer, num_incorrect_answers)
    return incorrect_answers

# Пример использования функции
incorrect_answers = create_answers('apple')
print(incorrect_answers)

#----------------------------------------------------------------------------
# 4. Формирование Ответов

# Функция для формирования ответов
def create_answers(question, correct_answer, context_sentences):
    wrong_answers = set()
    while len(wrong_answers) < 3:
        sample_sentence = random.choice(context_sentences)
        if correct_answer not in sample_sentence:
            wrong_answers.add(sample_sentence)
    wrong_answers = list(wrong_answers)

    full_question = {
        'question': question,
        'options': wrong_answers + [correct_answer],
        'correct_answer': correct_answer
    }
    return full_question
#--------------------------------------------------------
# 5. Создание полного квиза

# Функция для генерации полного квиза
def generate_full_quiz(questions):
    quiz = []
    for q in questions:
        context_sentences = q['context']
        correct_answer = q['correct_answer']
        question = q['question']
        full_question = create_answers(question, correct_answer, context_sentences)
        quiz.append(full_question)
    return quiz
#----------------------------------------------------------------------------------
# 6. Оценка правильности квиза

# Функция оценки ответов
def evaluate_quiz(quiz, user_answers):
    score = 0
    for question, user_answer in zip(quiz, user_answers):
        if user_answer == question['correct_answer']:
            score += 1
        else:
            score -= 0.5
    return score

# Основной блок кода
if __name__ == "__main__":
    # Путь к файлу презентации
    presentation_file_path = 'path_to_presentation.pptx'
    
    # Извлечение текста из презентации
    extracted_text = extract_text_from_presentation(presentation_file_path)
    
    # Создание вопросов викторины
    quiz_questions = create_quiz_questions(extracted_text)
    
    # Генерация полного квиза
    full_quiz = generate_full_quiz(quiz_questions)
    
    # Предполагаемые ответы пользователя (заполните своими данными)
    user_answers = ['ответ1', 'ответ2', 'ответ3', ...]
    
    # Оценка ответов пользователя
    final_score = evaluate_quiz(full_quiz, user_answers)
    
    # Вывод итогового счёта
    print(f"Ваш итоговый счёт: {final_score}")